# Subqueries Lab

In this lesson, we'll practice working with subqueries.  And we'll do so by again working with our dataset regarding different universities.

### Loading our Data

We can begin by loading our data with the following.

In [2]:
import pandas as pd

root = "https://raw.githubusercontent.com/data-eng-10-21/sql-subqueries/main"
salary_potential_df = pd.read_csv(f'{root}/school_prices/salary_potential.csv', index_col = 0)
tuition_cost_df = pd.read_csv(f'{root}/school_prices/tuition_cost.csv', index_col = 0)
diversity_df = pd.read_csv(f'{root}/school_prices/diversity_school.csv', index_col = 0)
tuition_income_df = pd.read_csv(f'{root}/school_prices/tuition_income.csv', index_col = 0)

Next, let's create our schools database.

In [3]:
import sqlite3
conn = sqlite3.connect('schools.db')

And now we can load the data into our database.

In [4]:
tuition_cost_df.to_sql('tuitions', conn, if_exists = 'replace')
salary_potential_df.to_sql('salaries', conn, if_exists = 'replace')
diversity_df.to_sql('diversity_categories', conn, if_exists = 'replace')
tuition_income_df.to_sql('tuition_incomes', conn, if_exists = 'replace')

### Exploring our data

Now, we have a couple of new tables in our database, so let's start by exploring them.  Let's start by using SQL to select the first three rows from the `diversity_categories` table.

In [6]:
sql = """
      SELECT *
      FROM diversity_categories
      LIMIT 3;
      """

pd.read_sql(sql, conn)

# 	name	total_enrollment	state	category	enrollment
# 0	University of Phoenix-Arizona	195059	Arizona	Women	134722
# 1	University of Phoenix-Arizona	195059	Arizona	American Indian / Alaska Native	876
# 2	University of Phoenix-Arizona	195059	Arizona	Asian	1959

,name,total_enrollment,state,category,enrollment
0,University of Phoenix-Arizona,195059,Arizona,Women,134722
1,University of Phoenix-Arizona,195059,Arizona,American Indian / Alaska Native,876
2,University of Phoenix-Arizona,195059,Arizona,Asian,1959


So we can see that each school has multiple entries, each for a different diversity category.  Next, let's write query, that selects just the entries where the category is `Women`.

> A subquery is not needed here -- a simple `where` statement will do.

In [21]:
sql = """
      SELECT *
      FROM diversity_categories
      WHERE category = 'Women'
      """

women_category_df = pd.read_sql(sql, conn)

In [22]:
women_category_df[:2]

,name,total_enrollment,state,category,enrollment
0,University of Phoenix-Arizona,195059,Arizona,Women,134722
1,Ivy Tech Community College-Central Indiana,91179,Indiana,Women,53476


Then write another select statement that only returns entries where the category is `Women`, but this time return columns for the name of the university -- aliased as college -- the state, and a `percentage_women` column.

> Once again, we do not need a subquery for this.

> To avoid getting zero, may need to cast each column as a float, [see reference](https://stackoverflow.com/questions/1666407/sql-server-division-returns-zero).

In [72]:
sql = """
      SELECT dc.name as college, dc.state, CAST(we.enrollment as float) / CAST(SUM(dc.enrollment) as float) as percentage_women 
      FROM diversity_categories dc       
      JOIN (
          SELECT *
          FROM diversity_categories
          WHERE category = 'Women'
          ) we ON dc.name = we.name
      WHERE dc.name LIKE 'Ivy%'
      GROUP BY 1
      """

In [82]:
sql = """
      SELECT name as college,  CAST(enrollment as float) / CAST(total_enrollment as float)
      as percentage_women 
      FROM diversity_categories WHERE category = 'Women'
      """

In [83]:
percentage_women_df = pd.read_sql(sql, conn)

In [85]:
percentage_women_df[:2]

# 	college	state	percentage_women
# 0	University of Phoenix-Arizona	Arizona	0.690673
# 1	Ivy Tech Community College-Central Indiana	Indiana	0.586495

,college,percentage_women
0,University of Phoenix-Arizona,0.690673
1,Ivy Tech Community College-Central Indiana,0.586495


Ok, now turn the query above into a subquery, and simply select the `college` and `percentage_women` columns from the subquery.  Alias the results of the subquery as `gender_splits`.

In [86]:
sql = """
      SELECT college, percentage_women 
      FROM (
        SELECT name as college,  CAST(enrollment as float) / CAST(total_enrollment as float)
        as percentage_women 
        FROM diversity_categories WHERE category = 'Women' ) AS gender_splits
"""

In [87]:
percentage_women_subquery_df = pd.read_sql(sql, conn)

percentage_women_subquery_df[:2]

,college,percentage_women
0,University of Phoenix-Arizona,0.690673
1,Ivy Tech Community College-Central Indiana,0.586495


### Joining a Table

Now let's work towards joining the results above with income information in the salaries table.

> First we can look at a couple of rows of salary data.

In [75]:
pd.read_sql("SELECT * FROM salaries LIMIT 2;", conn)

,rank,name,state_name,early_career_pay,mid_career_pay,make_world_better_percent,stem_percent
0,1,Auburn University,Alabama,54400,104500,51.0,31
1,2,University of Alabama in Huntsville,Alabama,57500,103900,59.0,45


Ok, so we can can begin by placing our entire previous query into a subquery. 

In [89]:
sql = """
SELECT * FROM 
(SELECT name as college,  CAST(enrollment as float) / CAST(total_enrollment as float)
as percentage_women 
FROM diversity_categories WHERE category = 'Women') as gender_splits;
"""

subquery_df = pd.read_sql(sql, conn)
subquery_df[:2]

,college,percentage_women
0,University of Phoenix-Arizona,0.690673
1,Ivy Tech Community College-Central Indiana,0.586495


And then because we can treat the subquery as a table `gender_splits` with columns of `college` and `percentage_women`, we can simply join the `salaries` table just like we would any other table.

In [90]:
sql = """
SELECT college, percentage_women, early_career_pay, mid_career_pay FROM 
(SELECT name as college,  CAST(enrollment as float) / CAST(total_enrollment as float)
as percentage_women 
FROM diversity_categories WHERE category = 'Women') as gender_splits 
INNER JOIN salaries ON salaries.name = gender_splits.college;
"""

joined_subquery_df = pd.read_sql(sql, conn)
joined_subquery_df[:2]

,college,percentage_women,early_career_pay,mid_career_pay
0,Auburn University,0.493902,54400,104500
1,Tuskegee University,0.597809,54500,93500


And from here, let's say find those schools where `mid_career_pay` is greater than 100,000 and sort by `percentage_women`.

In [92]:
sql = """
SELECT college, percentage_women, early_career_pay, mid_career_pay FROM 
(SELECT name as college,  CAST(enrollment as float) / CAST(total_enrollment as float)
as percentage_women 
FROM diversity_categories WHERE category = 'Women') as gender_splits 
INNER JOIN salaries ON salaries.name = gender_splits.college 
WHERE mid_career_pay > 100000 ORDER BY percentage_women DESC;
"""

joined_subquery_df = pd.read_sql(sql, conn)
joined_subquery_df[:5]

,college,percentage_women,early_career_pay,mid_career_pay
0,Barnard College,0.998057,59200,109800
1,Wellesley College,0.973741,58900,106200
2,Samuel Merritt University,0.741139,91200,154100
3,Rush University,0.731380,63500,107600
4,Texas Tech University Health Sciences Center,0.684648,61300,104900


### Your Turn

Ok, now it's your turn to use a subquery in a join.  We'll start you off with the subquery from before.

In [93]:
sql = """
SELECT * FROM 
(SELECT name as college,  CAST(enrollment as float) / CAST(total_enrollment as float)
as percentage_women 
FROM diversity_categories WHERE category = 'Women') as gender_splits ;
"""

subquery_df = pd.read_sql(sql, conn)
subquery_df[:2]

,college,percentage_women
0,University of Phoenix-Arizona,0.690673
1,Ivy Tech Community College-Central Indiana,0.586495


In [94]:
tuitions_df = pd.read_sql("SELECT * FROM tuitions LIMIT 5;", conn)
tuitions_df[:2]

,name,state,state_code,type,degree_length,room_and_board,in_state_tuition,in_state_total,out_of_state_tuition,out_of_state_total
0,Aaniiih Nakoda College,Montana,MT,Public,2 Year,NaN,2380,2380,2380,2380
1,Abilene Christian University,Texas,TX,Private,4 Year,10350.0,34850,45200,34850,45200


This time use JOIN to add the the `out_of_state_tuition` cost, aliased as tution along with the `college` and `percentage_women` columns.   Sort the results by the name of the college. 

In [96]:
sql = """
      SELECT gender_splits.*, out_of_state_tuition FROM 
        (SELECT name as college,  CAST(enrollment as float) / CAST(total_enrollment as float)
        as percentage_women 
        FROM diversity_categories WHERE category = 'Women') as gender_splits 
      JOIN tuitions ON tuitions.name = gender_splits.college
      ORDER BY college
      """

subquery_df = pd.read_sql(sql, conn)
subquery_df[:2]

# 	college	percentage_women	tuition
# 0	Aaniiih Nakoda College	0.611684	2380
# 1	Abilene Christian University	0.578721	34850

,college,percentage_women,out_of_state_tuition
0,Aaniiih Nakoda College,0.611684,2380
1,Abilene Christian University,0.578721,34850


Ok, now above, we have a select statement that returns a college, the percentage women, and tuition.  Let's add the average `net_cost` per college  these results.



To accomplish this, we need to load up some data from the `tuition_incomes` table.

In [98]:
sql = """
SELECT * FROM tuition_incomes;
"""
tuition_incomes_df = pd.read_sql(sql, conn)
tuition_incomes_df[:2]

,name,state,total_price,year,campus,net_cost,income_lvl
0,Piedmont International University,NC,20174,2016,On Campus,11475.0,"0 to 30,000"
1,Piedmont International University,NC,20174,2016,On Campus,11451.0,"30,001 to 48,000"


Now, because we want to keep the same dimensions of our earlier select statement (where only one record per each university), begin by writing a query to that returns, for each university, for their most recent year, the `average_net_cost` as well as the name of the school.

> Try not to refer to the previous reading.

In [100]:
sql = """
      SELECT name, MAX(year)
      FROM tuition_incomes
      GROUP BY name
"""

tuition_incomes_df = pd.read_sql(sql, conn)
tuition_incomes_df[:2]

,name,MAX(year)
0,AI Miami International University of Art and D...,2017
1,ASA College,2018


In [107]:
sql = """
      SELECT ti.name, ti.year, AVG(net_cost)
      FROM tuition_incomes ti
      JOIN (
        SELECT name, MAX(year) as myear
        FROM tuition_incomes
        GROUP BY 1
      ) AS max_year ON max_year.name = ti.name AND max_year.myear = ti.year
      GROUP BY 1, 2
      """

tuition_incomes_df = pd.read_sql(sql, conn)
tuition_incomes_df[:2]

# name	avg_net_cost	most_recent_year
# 0	AI Miami International University of Art and D...	23585.115344	2017
# 1	ASA College	27686.354078	2018

,name,year,AVG(net_cost)
0,AI Miami International University of Art and D...,2017,23585.115344
1,ASA College,2018,27686.354078


Now add this to our earlier sql queries to return the name of the `college`, `percentage_women`, and `avg_net_cost` for each college.

> You can see how to join on a subquery by referencing [the following](https://riptutorial.com/sql/example/9725/joining-on-a-subquery).

In [126]:
sql = """
      SELECT gender_splits.*, out_of_state_tuition, nc.avg_cost FROM 
        (SELECT name as college,  CAST(enrollment as float) / CAST(total_enrollment as float)
        as percentage_women 
        FROM diversity_categories WHERE category = 'Women') as gender_splits 
      JOIN tuitions ON tuitions.name = gender_splits.college
      JOIN (
          SELECT ti.name, ti.year, AVG(net_cost) avg_cost
          FROM tuition_incomes ti
          JOIN (
            SELECT name, MAX(year) as myear
            FROM tuition_incomes
            GROUP BY 1
          ) AS max_year ON max_year.name = ti.name AND max_year.myear = ti.year
          GROUP BY 1, 2) nc ON gender_splits.college = nc.name
      ORDER BY nc.name
"""

subquery_df = pd.read_sql(sql, conn)
subquery_df[:2]

# 	college	percentage_women	tuition	avg_net_cost
# 0	Aaniiih Nakoda College	0.611684	2380	5280.800000
# 1	Abilene Christian University	0.578721	34850	27520.337592

,college,percentage_women,out_of_state_tuition,avg_cost
0,Aaniiih Nakoda College,0.611684,2380,5280.800000
1,Abilene Christian University,0.578721,34850,27520.337592


### Summary

In this lesson, we practiced working with subqueries.  We saw how we can use subqueries to break our queries into steps, and also practiced joining on subqueries.